In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from matplotlib import pyplot


features_for_mood = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'instrumentalness', 'liveness', 'valence', 'tempo']

In [2]:
df = pd.read_csv('Full_spotify_data_senior_project64.csv')
df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,label
0,6s8JOwADhfEbqGqyKshMJE,0.386,0.500,2,-7.230,1,0.0371,0.0,0.0878,0.336,159.807,42.0,2013.0,sad
1,4N4VmSZhReQniqoakHBEvF,0.490,0.422,8,-7.775,1,0.0288,0.0,0.2240,0.348,129.772,39.0,2012.0,sad
2,7ETxkXmmEjVnloAXYzNUoH,0.818,0.634,8,-5.137,1,0.1260,0.0,0.1100,0.794,84.981,47.0,2013.0,sad
3,5AwQir16Hh3nHBLyGuhTjU,0.601,0.599,7,-5.549,1,0.0373,0.0,0.1530,0.577,67.998,49.0,2018.0,sad
4,6807mPDUKTRWDWiXaXBhke,0.699,0.594,0,-6.726,1,0.0418,0.0,0.0985,0.338,133.989,43.0,2007.0,sad


In [3]:
df.drop(columns=['danceability', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'tempo', 'popularity', 'year'])

,track_id,energy,valence,label
0,6s8JOwADhfEbqGqyKshMJE,0.500,0.336,sad
1,4N4VmSZhReQniqoakHBEvF,0.422,0.348,sad
2,7ETxkXmmEjVnloAXYzNUoH,0.634,0.794,sad
3,5AwQir16Hh3nHBLyGuhTjU,0.599,0.577,sad
4,6807mPDUKTRWDWiXaXBhke,0.594,0.338,sad
...,...,...,...,...
915,0JdDnGGlycKJr0iceYnvcK,0.731,0.398,cry
916,127FdW4XnmqKxRKrxRj2S5,0.666,0.321,cry
917,3Eu82PdSTJ8whj8KFJj4au,0.576,0.272,cry
918,4WUMYRlZFGppQbBQTgRb5O,0.558,0.628,cry


In [4]:
df.shape

(920, 14)

In [5]:
df_aggressive=df.loc[(df['energy']>=0.5) & (df['valence']<0.5)] 
df_aggressive['label']='aggressive'
df_aggressive

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,label
0,6s8JOwADhfEbqGqyKshMJE,0.386,0.500,2,-7.230,1,0.0371,0.000000,0.0878,0.336,159.807,42.0,2013.0,aggressive
4,6807mPDUKTRWDWiXaXBhke,0.699,0.594,0,-6.726,1,0.0418,0.000000,0.0985,0.338,133.989,43.0,2007.0,aggressive
5,3Zkc66ctXTrpk52tRcIoef,0.657,0.565,7,-7.696,1,0.0689,0.000299,0.0916,0.462,146.054,37.0,2013.0,aggressive
6,2osoiXSQrConE7kKdIlxin,0.444,0.649,9,-4.286,1,0.0273,0.000004,0.3600,0.310,146.092,45.0,2012.0,aggressive
8,3zc9UfgETcTMGiazW58Klg,0.473,0.848,4,-4.061,1,0.0296,0.000000,0.0991,0.396,160.012,51.0,2014.0,aggressive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,4cTUjZ61DVM4159Fet6Mee,0.637,0.659,2,-5.878,1,0.0419,0.000004,0.3920,0.370,133.965,37.0,2021.0,aggressive
915,0JdDnGGlycKJr0iceYnvcK,0.454,0.731,2,-5.374,1,0.0253,0.000007,0.1430,0.398,159.968,45.0,2020.0,aggressive
916,127FdW4XnmqKxRKrxRj2S5,0.598,0.666,5,-5.130,1,0.0339,0.000021,0.5180,0.321,157.955,9.0,2020.0,aggressive
917,3Eu82PdSTJ8whj8KFJj4au,0.522,0.576,7,-5.317,1,0.0257,0.000002,0.1160,0.272,74.268,44.0,2017.0,aggressive


In [6]:
df_aggressive.rename(columns={'label':'russel_label'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
df_aggressive.drop(columns=['danceability', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'tempo', 'popularity', 'year'])

,track_id,energy,valence,russel_label
0,6s8JOwADhfEbqGqyKshMJE,0.500,0.336,aggressive
4,6807mPDUKTRWDWiXaXBhke,0.594,0.338,aggressive
5,3Zkc66ctXTrpk52tRcIoef,0.565,0.462,aggressive
6,2osoiXSQrConE7kKdIlxin,0.649,0.310,aggressive
8,3zc9UfgETcTMGiazW58Klg,0.848,0.396,aggressive
...,...,...,...,...
912,4cTUjZ61DVM4159Fet6Mee,0.659,0.370,aggressive
915,0JdDnGGlycKJr0iceYnvcK,0.731,0.398,aggressive
916,127FdW4XnmqKxRKrxRj2S5,0.666,0.321,aggressive
917,3Eu82PdSTJ8whj8KFJj4au,0.576,0.272,aggressive


In [8]:
df_aggressive.shape

(216, 14)

In [9]:
df_happy=df.loc[(df['energy']>=0.5) & (df['valence']>=0.5)] 
df_happy['label']='happy'
df_happy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,label
2,7ETxkXmmEjVnloAXYzNUoH,0.818,0.634,8,-5.137,1,0.1260,0.000000,0.1100,0.794,84.981,47.0,2013.0,happy
3,5AwQir16Hh3nHBLyGuhTjU,0.601,0.599,7,-5.549,1,0.0373,0.000000,0.1530,0.577,67.998,49.0,2018.0,happy
7,24VfUtHxbkCrscWC00toom,0.501,0.648,0,-5.771,1,0.0272,0.000002,0.0913,0.504,180.043,51.0,2015.0,happy
17,4ywXyMFZ8cktvKIvJbiksy,0.753,0.649,5,-5.494,1,0.0742,0.000000,0.0778,0.550,98.952,39.0,2016.0,happy
18,6xxBWqtAELhcSt0kY2qo5Z,0.548,0.749,1,-4.620,1,0.0256,0.000000,0.2730,0.630,131.976,42.0,2017.0,happy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,5AwQir16Hh3nHBLyGuhTjU,0.601,0.599,7,-5.549,1,0.0373,0.000000,0.1530,0.577,67.998,49.0,2018.0,happy
898,2GQS4AkV1hCVSt0gPum44w,0.565,0.716,10,-6.704,1,0.0442,0.000000,0.0964,0.604,169.974,42.0,2017.0,happy
900,72KOmS4Af9IEFsT87TK1VQ,0.698,0.526,6,-6.289,0,0.1030,0.000095,0.1050,0.608,80.031,46.0,2017.0,happy
901,3armUxyNuJmexkQRA3hZSC,0.643,0.611,0,-7.634,1,0.0407,0.000000,0.1170,0.510,140.100,48.0,2021.0,happy


In [10]:
df_happy.rename(columns={'label':'russel_label'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
df_happy.drop(columns=['danceability', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'tempo', 'popularity', 'year'])

,track_id,energy,valence,russel_label
2,7ETxkXmmEjVnloAXYzNUoH,0.634,0.794,happy
3,5AwQir16Hh3nHBLyGuhTjU,0.599,0.577,happy
7,24VfUtHxbkCrscWC00toom,0.648,0.504,happy
17,4ywXyMFZ8cktvKIvJbiksy,0.649,0.550,happy
18,6xxBWqtAELhcSt0kY2qo5Z,0.749,0.630,happy
...,...,...,...,...
895,5AwQir16Hh3nHBLyGuhTjU,0.599,0.577,happy
898,2GQS4AkV1hCVSt0gPum44w,0.716,0.604,happy
900,72KOmS4Af9IEFsT87TK1VQ,0.526,0.608,happy
901,3armUxyNuJmexkQRA3hZSC,0.611,0.510,happy


In [12]:
df_happy.shape

(313, 14)

In [13]:
df_relax=df.loc[(df['energy']<0.5) & (df['valence']>=0.5)] 
df_relax['label']='relax'
df_relax

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,label
20,7IcfEY0IyV4fdALbwJCw3v,0.536,0.421,11,-11.907,0,0.0344,0.000112,0.1020,0.534,138.013,51.0,2019.0,relax
51,1OlInnJssr9pXY0Ooa0SDT,0.685,0.410,7,-9.358,1,0.0316,0.000000,0.1190,0.533,135.005,47.0,2017.0,relax
64,60hF3gZXQ33xdAXXk6tE1x,0.537,0.491,9,-9.060,1,0.0349,0.014000,0.1460,0.508,140.015,49.0,2017.0,relax
72,7EZnNHXfzf2C9fWU9pPDfN,0.678,0.328,5,-10.283,1,0.0315,0.000232,0.0987,0.543,143.940,36.0,2016.0,relax
80,18kwPdB3Bx8n9Mk4YsO6LT,0.662,0.438,7,-6.117,1,0.0342,0.000000,0.1310,0.514,79.924,0.0,2018.0,relax
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,6dcKIbnpi1CmMXUbrrXPUx,0.556,0.462,0,-6.259,1,0.0249,0.000000,0.1340,0.512,80.154,38.0,1987.0,relax
801,5T5YlY74jZuZVwWgFoNGJT,0.596,0.419,2,-10.827,1,0.0227,0.000000,0.1170,0.521,97.953,29.0,2013.0,relax
806,4MnjJ8vzq5GD3b7LGrwsyc,0.655,0.352,11,-14.298,1,0.0266,0.002080,0.2870,0.734,104.950,28.0,2013.0,relax
868,4vI3jw8Fi8fDH0gGWzttGt,0.617,0.461,3,-8.122,1,0.0299,0.000000,0.1610,0.641,74.984,37.0,2019.0,relax


In [14]:
df_relax.rename(columns={'label':'russel_label'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
df_relax.drop(columns=['danceability', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'tempo', 'popularity', 'year'])

,track_id,energy,valence,russel_label
20,7IcfEY0IyV4fdALbwJCw3v,0.421,0.534,relax
51,1OlInnJssr9pXY0Ooa0SDT,0.410,0.533,relax
64,60hF3gZXQ33xdAXXk6tE1x,0.491,0.508,relax
72,7EZnNHXfzf2C9fWU9pPDfN,0.328,0.543,relax
80,18kwPdB3Bx8n9Mk4YsO6LT,0.438,0.514,relax
...,...,...,...,...
785,6dcKIbnpi1CmMXUbrrXPUx,0.462,0.512,relax
801,5T5YlY74jZuZVwWgFoNGJT,0.419,0.521,relax
806,4MnjJ8vzq5GD3b7LGrwsyc,0.352,0.734,relax
868,4vI3jw8Fi8fDH0gGWzttGt,0.461,0.641,relax


In [16]:
df_relax.shape

(107, 14)

In [17]:
df_sad=df.loc[(df['energy']<0.5) & (df['valence']<0.5)] 
df_sad['label']='sad'
df_sad

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,label
1,4N4VmSZhReQniqoakHBEvF,0.490,0.422,8,-7.775,1,0.0288,0.000000,0.2240,0.3480,129.772,39.0,2012.0,sad
9,7N1mCWyLnYXfi3AXagJCNo,0.557,0.295,5,-12.706,1,0.0355,0.002010,0.1110,0.1330,76.061,51.0,2012.0,sad
11,7dfs4SVfLPVORzjmYYqorK,0.313,0.413,0,-8.255,1,0.0314,0.000000,0.3430,0.4520,207.865,38.0,2018.0,sad
13,1psb8glAxDb4DYAwnOWzm3,0.634,0.480,9,-9.913,1,0.0472,0.000048,0.1110,0.2750,128.046,50.0,2018.0,sad
14,5bipGzsqZoCEEN9It0OOXU,0.629,0.367,6,-10.521,1,0.0277,0.040300,0.1040,0.2910,94.031,38.0,2014.0,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,4WxNfBGTaGoN5hOqXI9TFJ,0.361,0.474,0,-8.155,1,0.0356,0.000404,0.0663,0.2490,170.018,46.0,2014.0,sad
906,1dexPsihTJqDO3QCzlZXdq,0.630,0.479,11,-7.371,1,0.0298,0.000000,0.0879,0.4600,140.120,44.0,2020.0,sad
911,4XwJ7O8RGVrKAdmEhYs8kZ,0.562,0.497,8,-7.974,1,0.0322,0.000000,0.1100,0.2210,82.100,8.0,2008.0,sad
913,0nedd58OQDiAUQHEJBrvcx,0.446,0.416,10,-8.256,1,0.0297,0.000000,0.1010,0.0661,91.621,38.0,2018.0,sad


In [18]:
df_sad.rename(columns={'label':'russel_label'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
df_sad.drop(columns=['danceability', 'key', 'loudness', 'mode', 'speechiness', 'instrumentalness', 'liveness', 'tempo', 'popularity', 'year'])

,track_id,energy,valence,russel_label
1,4N4VmSZhReQniqoakHBEvF,0.422,0.3480,sad
9,7N1mCWyLnYXfi3AXagJCNo,0.295,0.1330,sad
11,7dfs4SVfLPVORzjmYYqorK,0.413,0.4520,sad
13,1psb8glAxDb4DYAwnOWzm3,0.480,0.2750,sad
14,5bipGzsqZoCEEN9It0OOXU,0.367,0.2910,sad
...,...,...,...,...
899,4WxNfBGTaGoN5hOqXI9TFJ,0.474,0.2490,sad
906,1dexPsihTJqDO3QCzlZXdq,0.479,0.4600,sad
911,4XwJ7O8RGVrKAdmEhYs8kZ,0.497,0.2210,sad
913,0nedd58OQDiAUQHEJBrvcx,0.416,0.0661,sad


In [20]:
df_sad.shape

(284, 14)

In [21]:
df_newlabel=pd.concat([df_aggressive, df_happy,df_relax,df_sad])
df_newlabel

,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,popularity,year,russel_label
0,6s8JOwADhfEbqGqyKshMJE,0.386,0.500,2,-7.230,1,0.0371,0.000000,0.0878,0.3360,159.807,42.0,2013.0,aggressive
4,6807mPDUKTRWDWiXaXBhke,0.699,0.594,0,-6.726,1,0.0418,0.000000,0.0985,0.3380,133.989,43.0,2007.0,aggressive
5,3Zkc66ctXTrpk52tRcIoef,0.657,0.565,7,-7.696,1,0.0689,0.000299,0.0916,0.4620,146.054,37.0,2013.0,aggressive
6,2osoiXSQrConE7kKdIlxin,0.444,0.649,9,-4.286,1,0.0273,0.000004,0.3600,0.3100,146.092,45.0,2012.0,aggressive
8,3zc9UfgETcTMGiazW58Klg,0.473,0.848,4,-4.061,1,0.0296,0.000000,0.0991,0.3960,160.012,51.0,2014.0,aggressive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,4WxNfBGTaGoN5hOqXI9TFJ,0.361,0.474,0,-8.155,1,0.0356,0.000404,0.0663,0.2490,170.018,46.0,2014.0,sad
906,1dexPsihTJqDO3QCzlZXdq,0.630,0.479,11,-7.371,1,0.0298,0.000000,0.0879,0.4600,140.120,44.0,2020.0,sad
911,4XwJ7O8RGVrKAdmEhYs8kZ,0.562,0.497,8,-7.974,1,0.0322,0.000000,0.1100,0.2210,82.100,8.0,2008.0,sad
913,0nedd58OQDiAUQHEJBrvcx,0.446,0.416,10,-8.256,1,0.0297,0.000000,0.1010,0.0661,91.621,38.0,2018.0,sad


In [22]:
df_newlabel[['energy','valence','russel_label']]

,energy,valence,russel_label
0,0.500,0.3360,aggressive
4,0.594,0.3380,aggressive
5,0.565,0.4620,aggressive
6,0.649,0.3100,aggressive
8,0.848,0.3960,aggressive
...,...,...,...
899,0.474,0.2490,sad
906,0.479,0.4600,sad
911,0.497,0.2210,sad
913,0.416,0.0661,sad


In [23]:
df_newlabel['russel_label'].value_counts()

happy         313
sad           284
aggressive    216
relax         107
Name: russel_label, dtype: int64